In [1]:
import blingfire

In [4]:
from ctypes import *
import numpy as np

In [5]:
def wre(mweModel, wreModel, text, startOffsets, endOffsets, tags, maxOutputCount=None):
    if len(startOffsets) != len(endOffsets) or len(startOffsets) != len(tags):
        raise Exception("Invalid input")

    if maxOutputCount is None:
        # Default max output count to number of input words.
        maxOutputCount = len(tags)

    o_tags = (c_int32 * maxOutputCount)()
    o_froms = (c_int32 * maxOutputCount)()
    o_tos = (c_int32 * maxOutputCount)()

    text_bytes = text.encode("utf-8")

    outputCount = blingfire.blingfire.ParseWre(
        c_char_p(text_bytes),
        c_int(len(text_bytes)),
        c_void_p(startOffsets.__array_interface__['data'][0]),
        c_void_p(endOffsets.__array_interface__['data'][0]),
        c_void_p(tags.__array_interface__['data'][0]),
        c_int(len(tags)),
        byref(o_froms),
        byref(o_tos),
        byref(o_tags),
        c_int(maxOutputCount),
        c_void_p(mweModel),
        c_void_p(wreModel)
    )

    def intbuffer(buf):
        return np.frombuffer(
            buf, dtype=c_int32, count=min(outputCount, maxOutputCount)
        )

    return (
        intbuffer(o_froms),
        intbuffer(o_tos),
        intbuffer(o_tags),
        outputCount
    )
        
        